In [0]:
#load data and remove first few rows 
import pandas as pd 
data= pd.read_csv('/content/drive/My Drive/Colab Notebooks/unstructured data my project/covid19airquality2020.csv',skiprows=[0,1,2,3,4,5])
data2019q1= pd.read_csv('/content/drive/My Drive/Colab Notebooks/unstructured data my project/waqi-covid19-airqualitydata-2019Q1.csv',skiprows=[0,1,2,3,4,5])


In [21]:
data.head()

,Date,Country,City,Specie,count,min,max,median,variance
0,2020-03-18,MN,Ulan Bator,dew,253,-14.5,-4.0,-11.5,68.52
1,2020-03-27,MN,Ulan Bator,dew,264,-20.5,-14.5,-18.0,35.34
2,2020-04-04,MN,Ulan Bator,dew,264,-19.5,-10.0,-14.0,82.81
3,2020-04-09,MN,Ulan Bator,dew,264,-16.5,-11.0,-15.0,32.06
4,2020-01-29,MN,Ulan Bator,dew,240,-26.0,-15.0,-20.0,138.35


In [0]:
#create new csv with removed rows
data.to_csv(r'/content/drive/My Drive/Colab Notebooks/unstructured data my project/sirquality2020.csv', index = False, header =True)
data2019q1.to_csv(r'/content/drive/My Drive/Colab Notebooks/unstructured data my project/sirquality2019q1.csv', index = False, header =True)

In [2]:
#install pyspark 
pip install pyspark

     |████████████████████████████████| 217.8MB 58kB/s 
     |████████████████████████████████| 204kB 39.7MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=49d9421461e0ccbabc44d9e0460fcbdf7975f0012c3cb0a462c3b4525b61dd82
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark


In [0]:
#create spark session
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [0]:
from pyspark import SQLContext
sqlContext= SQLContext(sc)

In [0]:
#load data with spark 
data20 = sqlContext.read.csv('/content/drive/My Drive/Colab Notebooks/unstructured data my project/sirquality2020.csv',header= True)
data19 = sqlContext.read.csv('/content/drive/My Drive/Colab Notebooks/unstructured data my project/sirquality2019q1.csv',header=True)

Looking at the data 

In [48]:
data20.show()

+----------+-------+----------+------+-----+-----+-----+------+--------+
|      Date|Country|      City|Specie|count|  min|  max|median|variance|
+----------+-------+----------+------+-----+-----+-----+------+--------+
|2020-03-18|     MN|Ulan Bator|   dew|  253|-14.5| -4.0| -11.5|   68.52|
|2020-03-27|     MN|Ulan Bator|   dew|  264|-20.5|-14.5| -18.0|   35.34|
|2020-04-04|     MN|Ulan Bator|   dew|  264|-19.5|-10.0| -14.0|   82.81|
|2020-04-09|     MN|Ulan Bator|   dew|  264|-16.5|-11.0| -15.0|   32.06|
|2020-01-29|     MN|Ulan Bator|   dew|  240|-26.0|-15.0| -20.0|  138.35|
|2020-02-07|     MN|Ulan Bator|   dew|  240|-32.0|-14.0| -24.0|   325.9|
|2020-02-27|     MN|Ulan Bator|   dew|  264|-25.0|-11.5| -16.0|  145.02|
|2020-03-05|     MN|Ulan Bator|   dew|  264|-24.0|-10.0| -19.0|  204.31|
|2020-03-21|     MN|Ulan Bator|   dew|  264|-13.0| -8.0| -11.0|   28.61|
|2020-01-05|     MN|Ulan Bator|   dew|  240|-25.0|-15.5| -19.0|   55.12|
|2020-02-09|     MN|Ulan Bator|   dew|  140|-26.0|-

In [98]:
data20.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Specie: string (nullable = true)
 |-- count: string (nullable = true)
 |-- min: string (nullable = true)
 |-- max: string (nullable = true)
 |-- median: string (nullable = true)
 |-- variance: string (nullable = true)



Some simple Aggregations 

In [0]:
#building a temporary table 
data20.registerTempTable("data20")
data19.registerTempTable("data19")

In [0]:
SpecieTpyeCount20 = sqlContext.sql("""
                           SELECT Specie, count(*) as freq
                           FROM data20
                           GROUP BY Specie
                           ORDER BY 2 DESC
                           """)

In [81]:
SpecieTpyeCount20.show()

+-------------+-----+
|       Specie| freq|
+-------------+-----+
|         pm25|53159|
|  temperature|51976|
|     humidity|51967|
|     pressure|51809|
|         pm10|51795|
|          no2|51583|
|           o3|47829|
|   wind speed|44075|
|          so2|43097|
|           co|39727|
|          dew|39640|
|    wind gust|28267|
|   wind-speed|10518|
|    wind-gust| 5864|
|precipitation| 4141|
|           wd| 3918|
|          aqi| 1512|
|          uvi| 1053|
|          pol|  715|
|          pm1|  413|
+-------------+-----+
only showing top 20 rows



In [0]:
MedianSpecieperCity = sqlContext.sql("""
                           SELECT City,Specie, sum(Median) as sum
                           FROM data20
                           GROUP BY City,Specie
                           ORDER BY 2 DESC
                           """)

In [117]:
MedianSpecieperCity.show()

+-------------------+----------+------------------+
|               City|    Specie|               sum|
+-------------------+----------+------------------+
|       Buenos Aires|wind-speed|              64.2|
|           Hannover|wind-speed|              28.4|
|             Riyadh|wind-speed|              58.6|
|               Yazd|wind-speed|              47.5|
|          São Paulo|wind-speed|              45.9|
|    Pachuca de Soto|wind-speed|             108.0|
|          Bucharest|wind-speed|48.199999999999996|
|              Rouen|wind-speed|              53.8|
|Ecatepec de Morelos|wind-speed|               8.0|
|            Wrocław|wind-speed|              44.1|
|              Wuhan|wind-speed|              35.2|
|             Berlin|wind-speed|              72.8|
|          Stockholm|wind-speed| 77.29999999999998|
|           Limassol|wind-speed| 82.99999999999997|
|             Odessa|wind-speed|              66.5|
|         Nur-Sultan|wind-speed| 88.19999999999997|
|           

In [118]:
# filter data to remove some weather measurements that we dont care about 
data20_filtered=data20.filter((data20.Specie=="no2")| (data20.Specie=="pm10")| (data20.Specie=="co")|(data20.Specie=="pm25")| (data20.Specie=="o3")| (data20.Specie=="so2"))
data20_filtered.show()

+----------+-------+----------+------+-----+---+----+------+--------+
|      Date|Country|      City|Specie|count|min| max|median|variance|
+----------+-------+----------+------+-----+---+----+------+--------+
|2020-02-18|     MN|Ulan Bator|   so2|  165|5.6|58.6|  18.8| 1491.11|
|2020-02-17|     MN|Ulan Bator|   so2|  180|6.6|58.6|  19.9| 2135.84|
|2020-03-18|     MN|Ulan Bator|   so2|  168|0.6|14.2|   4.1|   89.78|
|2020-01-25|     MN|Ulan Bator|   so2|  165|4.6|65.9|  14.7| 2602.46|
|2020-02-16|     MN|Ulan Bator|   so2|  186|7.1|58.8|  18.8| 2298.95|
|2020-03-13|     MN|Ulan Bator|   so2|  216|0.6|40.8|  10.3|  706.04|
|2020-03-17|     MN|Ulan Bator|   so2|  212|1.1|45.1|   8.2|  557.59|
|2020-01-17|     MN|Ulan Bator|   so2|  165|6.6|62.4|  20.3| 2976.86|
|2020-01-23|     MN|Ulan Bator|   so2|  158|6.3|52.3|  20.1| 1330.57|
|2020-04-02|     MN|Ulan Bator|   so2|  227|0.6|43.9|  10.2|  641.51|
|2020-01-13|     MN|Ulan Bator|   so2|  181|7.6|79.0|  16.9|  4623.9|
|2020-02-19|     MN|